In [1]:
import argparse
import os

import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import cifar10
from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten,
                          MaxPooling3D)
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import numpy as np
import cv2

Using TensorFlow backend.


In [11]:
batch = 15
epoch = 50
videos = "/Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw"
#videos = "/Users/gautamkarnataki/MIDS/w251/project/3dcnn/data/video_NEW3"
nclass = 19
output = "/Users/gautamkarnataki/MIDS/w251/project/3dcnn/3DCNN/3dcnnresult"
color = False
skip = False
depth = 10

In [12]:
class Videoto3D:

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

    def video3d(self, filename, color=False, skip=True):
        print('filename from videocapture = ', filename)
        cap = cv2.VideoCapture(filename)
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        if skip:
            frames = [x * nframe / self.depth for x in range(self.depth)]
        else:
            frames = [x for x in range(self.depth)]
        framearray = []

        for i in range(self.depth):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
            ret, frame = cap.read()
            if ret:
                frame = cv2.resize(frame, (self.height, self.width))
                if color:
                    framearray.append(frame)
                else:
                    framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()
        return np.array(framearray)


    def get_UCF_classname(self, filename):
        string = filename[filename.find('-',3)+9:]
        #print("class =========>", string[:string.find('_')])
        return string[:string.find('_')]

In [13]:
def loaddata(video_dir, vid3d, nclass, result_dir, color=False, skip=True):
    print('Inside load data -------------------------------')
    files = os.listdir(video_dir)
    X = []
    labels = []
    labellist = []

    pbar = tqdm(total=len(files))

    for filename in files:
        pbar.update(1)
        if filename == '.DS_Store':
            continue
        name = os.path.join(video_dir, filename)
        label = vid3d.get_UCF_classname(filename)
        if label not in labellist:
            if len(labellist) >= nclass:
                continue
            labellist.append(label)
        labels.append(label)
        X.append(vid3d.video3d(name, color=color, skip=skip))

    pbar.close()
    print("result_dir==============>", result_dir)
    with open(os.path.join(result_dir, 'classes.txt'), 'w') as fp:
        for i in range(len(labellist)):
            print("label ============================= ", labellist[i])
            fp.write('{}\n'.format(labellist[i]))

    for num, label in enumerate(labellist):
        for i in range(len(labels)):
            if label == labels[i]:
                labels[i] = num
    if color:
        return np.array(X).transpose((0, 2, 3, 4, 1)), labels
    else:
        return np.array(X).transpose((0, 2, 3, 1)), labels

In [29]:
img_rows, img_cols, frames = 32, 32, depth
channel = 15 if color else 1
vid3d = Videoto3D(img_rows, img_cols, frames)
nb_classes = nclass
x, y = loaddata(videos, vid3d, nclass, output, color, skip)
X = x.reshape((x.shape[0], img_rows, img_cols, frames, channel))
Y = np_utils.to_categorical(y, nb_classes)
X = X.astype('float32')
    #np.savez(fname_npz, X=X, Y=Y)
print('Saved dataset to dataset.npz.')
print('X_shape:{}\nY_shape:{}'.format(X.shape, Y.shape))



  0%|          | 0/156 [00:00<?, ?it/s]

  2%|▏         | 3/156 [00:00<00:05, 25.89it/s]

Inside load data -------------------------------
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2011_06_08_Brady_scene33-camera1_DEPOSIT_v0_4_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2011_06_08_Brady_scene25-camera1_CAN_v0_4_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2011_06_08_Brady_scene29-camera1_CONFLICT_v0_4_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2007_05_24_scene11-camera1_DOCTOR_v1_7_train.mov




  3%|▎         | 5/156 [00:00<00:07, 21.09it/s]

  4%|▍         | 7/156 [00:00<00:08, 18.55it/s]

filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2008_01_11_scene81-camera1_COAT_v0_0_val.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2008_05_12b_scene2-camera1_CONFLICT_v0_2_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2008_01_18_scene24-camera1_DRESS_v1_3_val.mov




  6%|▌         | 9/156 [00:00<00:08, 17.12it/s]

...

filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2011_06_14_Brady_scene43-camera1_COURT_v0_6_train.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2008_02_01_scene38-camera1_COURT_v0_1_val.mov




 99%|█████████▊| 154/156 [00:34<00:00,  4.97it/s]

 99%|█████████▉| 155/156 [00:34<00:00,  5.65it/s]

filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2008_05_29a_scene4-camera1_DRESS_v2_9_val.mov
filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2008_05_12a_scene44-camera1_CHEAT_v2_1_val.mov




100%|██████████| 156/156 [00:34<00:00,  4.54it/s]

filename from videocapture =  /Users/gautamkarnataki/MIDS/w251/project/w251-project/data/raw/ASL_2006_10_10_scene5-camera1_CHEAP_v0_4_train.mov
result_dir==============> /Users/gautamkarnataki/MIDS/w251/project/3dcnn/3DCNN/3dcnnresult
label =============================  DEPOSIT
label =============================  CAN
label =============================  CONFLICT
label =============================  DOCTOR
label =============================  COAT
label =============================  DRESS
label =============================  CHAT
label =============================  ENOUGH
label =============================  WANT
label =============================  ALL
label =============================  BEHAVIOR
label =============================  BASEBALL
label =============================  CHEAT
label =============================  AWKWARD
label =============================  DEPRESS
label =============================  AGAIN
label =============================  COURT
label ==================

In [24]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD

In [30]:
# Define model
#baseModel = ResNet50(weights="imagenet", include_top=False)
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(X.shape[1:]), border_mode='same'))
model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss=categorical_crossentropy,optimizer=Adam(), metrics=['accuracy'])
model.summary()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=43)

for i in range(50):
    print('Y train =', Y_train[i])

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=batch,
                    epochs=epoch, verbose=1, shuffle=True)
model.evaluate(X_test, Y_test, verbose=0)
model_json = model.to_json()
if not os.path.isdir(output):
    os.makedirs(output)
with open(os.path.join(output, 'ucf101_3dcnnmodel.json'), 'w') as json_file:
    json_file.write(model_json)
model.save_weights(os.path.join(output, 'ucf101_3dcnnmodel.hd5'))

loss, acc = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', acc)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, kernel_size=(3, 3, 3), input_shape=(32, 32, 1..., padding="same")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, kernel_size=(3, 3, 3), padding="same")`
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(3, 3, 3), padding="same")`
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, kernel_size=(3, 3, 3), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, kernel_size=(3, 3, 3), pad

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 32, 32, 10, 32)    896       
_________________________________________________________________
activation_348 (Activation)  (None, 32, 32, 10, 32)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 32, 32, 10, 32)    27680     
_________________________________________________________________
activation_349 (Activation)  (None, 32, 32, 10, 32)    0         
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 11, 11, 4, 32)     0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 11, 11, 4, 32)     0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 11, 11, 4, 64)    

124/124 [==============================] - 13s 106ms/step - loss: 2.8696 - accuracy: 0.1290 - val_loss: 3.0267 - val_accuracy: 0.1250
Epoch 17/50
124/124 [==============================] - 13s 102ms/step - loss: 2.9418 - accuracy: 0.0968 - val_loss: 3.0046 - val_accuracy: 0.1250
Epoch 18/50
124/124 [==============================] - 14s 111ms/step - loss: 2.9538 - accuracy: 0.1129 - val_loss: 3.0405 - val_accuracy: 0.1250
Epoch 19/50
124/124 [==============================] - 14s 111ms/step - loss: 2.8675 - accuracy: 0.1290 - val_loss: 2.9859 - val_accuracy: 0.1250
Epoch 20/50
124/124 [==============================] - 13s 102ms/step - loss: 2.9176 - accuracy: 0.1048 - val_loss: 3.0219 - val_accuracy: 0.1250
Epoch 21/50
124/124 [==============================] - 13s 107ms/step - loss: 2.8546 - accuracy: 0.1290 - val_loss: 3.0221 - val_accuracy: 0.1250
Epoch 22/50
124/124 [==============================] - 12s 100ms/step - loss: 2.8689 - accuracy: 0.1290 - val_loss: 2.9915 - val_accurac